In [4]:
#!/usr/bin/env python3
# =====================================================================
#  C4 – Post-Steady-State Stability  (self-contained, 5⁵ grid)
# ---------------------------------------------------------------------
#  Verifies Conclusion (C4):
#     After a 10-s, ±5×10⁻⁴ rad plateau, δ(t) stays essentially
#     constant for the remainder of a 100-s simulation.
#
#  Parameter grid (5 values on each axis → 5⁵ = 3 125 trajectories)
#  ─────────────────────────────────────────────────────────────────
#     δ₀ : linspace(-π,  π, 5)
#     p₁ : linspace(0,   2, 5)
#     p₂ : linspace(0,   2, 5)
#     a  : {1, 2, 3, 4, 5}
#     b  : {1, 2, 3, 4, 5}
#
#  Numerical setup
#  ---------------
#     • Time span 0–100 s,  output step 0.001 s
#     • Plateau: 10-s window, ±5×10⁻⁴ rad band
#     • Tail tolerance (post-plateau): 1×10⁻³ rad
#
#  Output (console only)
#  ---------------------
#     trajectories integrated / stable / drift / no-plateau counts
#     pass-rate among plateaued cases
#     worst |δ − δ⋆| in plateaued set
# =====================================================================

import numpy as np
from math import sin, cos, pi
from itertools import product
from multiprocessing import Pool, cpu_count
from scipy.integrate import solve_ivp
from tqdm import tqdm

# ---------- numerical constants --------------------------------------
T_MAX, DT   = 100.0, 0.001
TIME_EVAL   = np.arange(0.0, T_MAX + DT, DT)

PLAT_WIN_S  = 10.0
PLAT_WIN_N  = int(PLAT_WIN_S / DT)        # 10 000 points
PLAT_TOL    = 5e-4                        # plateau band (rad)
TAIL_TOL    = 1e-3                        # allowed tail drift (rad)

# ---------- parameter grid (5 values each) ---------------------------
delta0_vals = np.linspace(-pi,  pi, 5)
p1_vals     = np.linspace(0.0, 2.0, 5)
p2_vals     = np.linspace(0.0, 2.0, 5)
a_vals      = np.array([1, 2, 3, 4, 5], dtype=float)
b_vals      = np.array([1, 2, 3, 4, 5], dtype=float)

GRID        = list(product(delta0_vals, p1_vals, p2_vals, a_vals, b_vals))
TOTAL_JOBS  = len(GRID)                   # 3 125

# ---------- ODE definition -------------------------------------------
def ode(t, y, p1, p2, a, b):
    θ, ψ = y
    s2   = sin(2*(ψ - θ))
    s, c = sin(ψ - θ), cos(ψ - θ)
    denom = 2 * ((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b) * (a**2 - b**2) * s2
    dθ_dt = -1.0 - p1*num/denom
    dψ_dt = -p2*s2
    return [dθ_dt, dψ_dt]

# ---------- plateau finder -------------------------------------------
def first_plateau(series):
    for i in range(len(series) - PLAT_WIN_N):
        win = series[i : i + PLAT_WIN_N]
        if np.max(np.abs(win - win[0])) < PLAT_TOL:
            return i, win[0]
    return None, None

# ---------- worker ---------------------------------------------------
def assess(params):
    δ0, p1, p2, a, b = params
    θ0, ψ0           = pi/3, pi/3 + δ0

    sol = solve_ivp(lambda t, y: ode(t, y, p1, p2, a, b),
                    (0.0, T_MAX), [θ0, ψ0],
                    t_eval=TIME_EVAL, rtol=1e-6, atol=1e-9)
    if not sol.success:
        return "solver_fail", np.nan

    δ = sol.y[1] - sol.y[0]
    idx, δ_star = first_plateau(δ)
    if idx is None:
        return "no_plateau", np.nan

    tail   = δ[idx + PLAT_WIN_N :]
    maxdev = float(np.max(np.abs(tail - δ_star)))
    return ("stable" if maxdev <= TAIL_TOL else "drift", maxdev)

# ---------- main -----------------------------------------------------
def main():
    with Pool(cpu_count()) as pool:
        results = list(
            tqdm(pool.imap_unordered(assess, GRID),
                 total=TOTAL_JOBS, desc="Integrating")
        )

    status, dev = zip(*results)
    total   = TOTAL_JOBS
    stable  = status.count("stable")
    drift   = status.count("drift")
    noplat  = status.count("no_plateau")
    worst   = np.nanmax(dev) if stable or drift else np.nan

    print("\nPost-steady-state stability summary (5⁵ grid)")
    print("──────────────────────────────────────────────")
    print(f" trajectories integrated : {total}")
    print(f"   stable                : {stable}")
    print(f"   drift                 : {drift}")
    print(f"   no plateau            : {noplat}")
    if stable + drift:
        pct = 100 * stable / (stable + drift)
        print(f" pass rate (plateaued)   : {pct:.2f}%")
        print(f" worst |δ − δ⋆|          : {worst:.3e} rad")

if __name__ == "__main__":
    main()


Integrating: 100%|██████████| 3125/3125 [01:40<00:00, 31.07it/s]



Post-steady-state stability summary (5⁵ grid)
──────────────────────────────────────────────
 trajectories integrated : 3125
   stable                : 2180
   drift                 : 0
   no plateau            : 945
 pass rate (plateaued)   : 100.00%
 worst |δ − δ⋆|          : 7.809e-04 rad
